# Классификация комментариев

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Цель-и-план-работы" data-toc-modified-id="Цель-и-план-работы-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Цель и план работы</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Лемматизация-и-очистка-текста." data-toc-modified-id="Лемматизация-и-очистка-текста.-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Лемматизация и очистка текста.</a></span></li><li><span><a href="#Расчёт-TF-IDF" data-toc-modified-id="Расчёт-TF-IDF-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Расчёт TF-IDF</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#RandomForestClassifier" data-toc-modified-id="RandomForestClassifier-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>RandomForestClassifier</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Анализ моделей</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Цель и план работы

**Задача** 

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

В нашем распоряжении набор данных с разметкой о токсичности правок.  
Необходимо обучить модель классифицировать комментарии на позитивные и негативные. 

Постройте модель со значением метрики качества *F1* не меньше 0.75.  

**Описание данных**

В нашем распоряжении набор данных с разметкой о токсичности правок.  
Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**План работы**

1. Загрузить и подготовить данные.
2. Обучить разные модели. 
3. Сделайть выводы.

## Подготовка данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np

import spacy
import re

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

from tqdm.notebook import tqdm

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

### Обзор данных

Загрузим и изучим данные.

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


В датафрейме нет пропусков. Проверим датафрейм на дубликаты строк.

In [4]:
df['text'].duplicated().sum()

0

Дубликаты отсутствуют. Изучим балансировку классов.

In [5]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Удалим незначимы столбец `Unnamed: 0`.

In [6]:
df = df.drop('Unnamed: 0', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


**Вывод**  
1) В датафрейме 159292 комментариев  
2) Комментарии на английском языке  
3) Пропуски данных и дубликаты строк отсутствуют    
4) Балансировка классов целевого признака: 10% негативных комментариев, 90% позитивных комментариев  
5) Незначимый столбец `Unnamed: 0` был удалён.

### Лемматизация и очистка текста.

Напишем функцию `lemmatize` для очистки и лемматизации текста с помощью `spaCy`.

In [7]:
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize_text(text):
    text = text.lower()
    cleared_text = " ".join(re.sub(r'[^a-zA-Z]', ' ', text).split())
    doc = lemmatizer(cleared_text)
    lemm_text = ' '.join([token.lemma_ for token in doc])
    return lemm_text

Сравним текст до и после лемматизации.

In [8]:
df['text'].iloc[2]

"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."

In [9]:
lemmatize_text(df['text'].iloc[2])

'hey man I m really not try to edit war it s just that this guy be constantly remove relevant information and talk to I through edit instead of my talk page he seem to care more about the formatting than the actual info'

Лемматизация текста прошла успешна. Применим лемматизацию для всего датафрейма.

In [10]:
%%time

tqdm.pandas()
df['lemm_text'] = df['text'].progress_apply(lemmatize_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

CPU times: user 16min 36s, sys: 9.97 s, total: 16min 46s
Wall time: 17min 17s


In [11]:
df['lemm_text'].head()

0    explanation why the edit make under my usernam...
1    d aww he match this background colour I m seem...
2    hey man I m really not try to edit war it s ju...
3    more I can t make any real suggestion on impro...
4    you sir be my hero any chance you remember wha...
Name: lemm_text, dtype: object

### Расчёт TF-IDF

Разделим датафрейм на обучающую и тестовую выборки в соотношении 90% / 10% соответственно.

In [12]:
X = df['lemm_text']
y = df['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

X_train.shape, X_test.shape

((143362,), (15930,))

Вычислим TF-IDF для корпуса текстов с комментариями.

In [13]:
%%time

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

tf_idf = TfidfVectorizer(stop_words=stopwords)
X_tf_idf_train = tf_idf.fit_transform(X_train.values)
X_tf_idf_test = tf_idf.transform(X_test.values)

X_tf_idf_train.shape, X_tf_idf_test.shape

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


CPU times: user 6.94 s, sys: 124 ms, total: 7.07 s
Wall time: 7.28 s


((143362, 142503), (15930, 142503))

Получили обучающую и тестовую матрицы TF-IDF по корпусу комментариев.

## Обучение моделей

Оценим качество моделей логистической регрессии, дерева решений, случайного леса, CatBoostClassifier.  
Так как классы целевого признака несбалансированны, будем использовать балансировку классов по весу.

Напишем функцию для поиска гиперпараметров по сетке GridSearchCV для наилучшей модели по метрике F1.

In [14]:
def get_hyperparams_and_f1(clf, hyperparams, cv_counts, model_name):
    
    pipe = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('clf', clf)])
    
    grid = GridSearchCV(estimator=pipe, param_grid=hyperparams, scoring='f1', cv=cv_counts)
    grid.fit(X_train, y_train)
    
    print(f'Гиперпараметры наилучшей модели {model_name}: {grid.best_params_}')
    
    best_f1 = grid.best_score_.round(4)
    print(f'Наилучший F1 {model_name} на кросс-валидации: {best_f1}')
    
    return best_f1, grid.best_params_

### LogisticRegression

In [15]:
%%time

clf = LogisticRegression()
hyperparams = [{'clf__solver':['newton-cg', 'lbfgs', 'liblinear'],
                'clf__C':[0.1, 1, 10],
                'clf__class_weight':['balanced'],
                'clf__random_state':[12345]}]
F1_LR_cv, LR_best_params = get_hyperparams_and_f1(clf, hyperparams, 2, 'LogisticRegression')

Гиперпараметры наилучшей модели LogisticRegression: {'clf__C': 10, 'clf__class_weight': 'balanced', 'clf__random_state': 12345, 'clf__solver': 'lbfgs'}
Наилучший F1 LogisticRegression на кросс-валидации: 0.7679
CPU times: user 4min 50s, sys: 3min 13s, total: 8min 4s
Wall time: 8min 5s


### DecisionTreeClassifier

In [16]:
%%time

clf = DecisionTreeClassifier()
hyperparams = [{'clf__max_depth': range(20, 101, 20),
                'clf__class_weight':['balanced'],
                'clf__random_state':[12345]}]
F1_DT_cv, DT_best_params = get_hyperparams_and_f1(clf, hyperparams, 2, 'DecisionTreeClassifier')

Гиперпараметры наилучшей модели DecisionTreeClassifier: {'clf__class_weight': 'balanced', 'clf__max_depth': 80, 'clf__random_state': 12345}
Наилучший F1 DecisionTreeClassifier на кросс-валидации: 0.6562
CPU times: user 6min 17s, sys: 649 ms, total: 6min 18s
Wall time: 6min 18s


### RandomForestClassifier

In [20]:
%%time

clf = RandomForestClassifier()
hyperparams = [{'clf__n_estimators': range(20, 101, 20),
                'clf__max_depth': range(20, 101, 20),
                'clf__class_weight':['balanced'],
                'clf__random_state':[12345]}]
F1_RF_cv, RF_best_params = get_hyperparams_and_f1(clf, hyperparams, 2, 'RandomForestClassifier')

Гиперпараметры наилучшей модели RandomForestClassifier: {'clf__class_weight': 'balanced', 'clf__max_depth': 100, 'clf__n_estimators': 80, 'clf__random_state': 12345}
Наилучший F1 RandomForestClassifier на кросс-валидации: 0.5409
CPU times: user 26min 46s, sys: 3.24 s, total: 26min 49s
Wall time: 26min 53s


### CatBoostClassifier

In [21]:
%%time

clf = CatBoostClassifier()
hyperparams = [{'clf__learning_rate': [0.5],
                'clf__depth': [3, 5, 7],
                'clf__verbose': [False],
                'clf__iterations': [250],
                'clf__random_state':[12345]}]
F1_CB_cv, CB_best_params = get_hyperparams_and_f1(clf, hyperparams, 2, 'CatBoostClassifier')

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Гиперпараметры наилучшей модели CatBoostClassifier: {'clf__depth': 5, 'clf__iterations': 250, 'clf__learning_rate': 0.5, 'clf__random_state': 12345, 'clf__verbose': False}
Наилучший F1 CatBoostClassifier на кросс-валидации: 0.7568
CPU times: user 40min 39s, sys: 23.8 s, total: 41min 2s
Wall time: 41min 20s


## Анализ моделей

Сведем результаты качества моделей в таблицу.

In [22]:
index = ['LogisticRegression',
         'DecisionTreeClassifier',
         'RandomForestRClassifier',
         'CatBoostClassifier']
data = {'F1_on_cv':[F1_LR_cv,
                    F1_DT_cv,
                    F1_RF_cv,
                    F1_CB_cv]
       }

scores_data = pd.DataFrame(data=data, index=index)
scores_data.sort_values(by='F1_on_cv', ascending=False)

,F1_on_cv
LogisticRegression,0.7679
CatBoostClassifier,0.7568
DecisionTreeClassifier,0.6562
RandomForestRClassifier,0.5409


Лучшей моделью является модель LogisticRegression.

## Тестирование

Проверим качество наилучшей модели на тестовой выборке.

In [23]:
#Модифицируем ключи словаря с наилучшими гиперпараметрами модели из GridSearch: удаление pipeline приставки 'clf__'
LR_best_params_modified = {}
for key, value in LR_best_params.items():
    new_key = key.replace('clf__', '')
    LR_best_params_modified[new_key] = value
    
LR_best_params_modified

{'C': 10, 'class_weight': 'balanced', 'random_state': 12345, 'solver': 'lbfgs'}

In [24]:
%%time

model = LogisticRegression()
model.set_params(**LR_best_params_modified)
model.fit(X_tf_idf_train, y_train)
predictons_test = model.predict(X_tf_idf_test)
print(f'F1 модели LogisticRegression на тестовой выборке: {f1_score(y_test, predictons_test).round(4)}')

F1 модели LogisticRegression на тестовой выборке: 0.7575
CPU times: user 20.4 s, sys: 29.1 s, total: 49.5 s
Wall time: 49.5 s


## Выводы

В данной работе была построена модель для классификации комментариев на позитивные и негативные.  

Нам был предоставлен исходный датасет. В результате обзора и подготовки данных были сделаны следующие преобразования и выводы:   
1) В датафрейме 159292 комментариев  
2) Комментарии на английском языке  
3) Пропуски данных и дубликаты строк отсутствуют    
4) Балансировка классов целевого признака: 10% негативных комментариев, 90% позитивных комментариев  
5) Незначимый столбец `Unnamed: 0` был удалён  
6) Была проведена очистка и лемматизация текста с помощью `spaCy`  
7) Рассчитаны TF-IDF по корпусу комментариев.

Далее были обучены 4 модели для сравнения качества по метрике F1 на валидационной выборке:  

1) LogisticRegression  
2) DecisionTreeClassifier  
3) RandomForestClassifier  
4) CatBoostClassifier  

Сравнительный анализ показал, что наилучшей моделью по метрике F1 для классификации комментариев на позитивные и негативные является модель LogisticRegression. Модель показала приемлемый результат качества на тестовой выборке.